In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
model_name = 'SM_with_load'
dynamic = False
load_type = 'const_Z'
V_gen_pu = None
if not dynamic and load_type != 'const_Z':
    raise Exception('Load type must be "const_Z" if load is static')
expt_name = '{}_{}_{}'.format('dynamic' if dynamic else 'static',
                              'loads' if 'loads' in model_name else 'load',
                              load_type)
if dynamic and load_type == 'const_Z' and V_gen_pu is not None:
    expt_name += '_Vgen={:.2f}_pu'.format(V_gen_pu)
folder = os.path.join('..','..','modal_analysis',model_name,expt_name)
filename = os.path.join(folder, 'SM_with_load_tran.npz')
blob = np.load(filename, allow_pickle=True)
data = blob['data'].item()
time = blob['time']
ur,ui = data['load']['m:u1r:bus1'], data['load']['m:u1i:bus1']
U,Ul = data['load']['m:U1:bus1'], data['load']['m:U1l:bus1']
phiu = data['load']['m:phiu1:bus1']
ir,ii = data['load']['m:i1r:bus1'], data['load']['m:i1i:bus1']
I = data['load']['m:I1:bus1']
phii = data['load']['m:phii1:bus1']
u = np.sqrt(3) * U * np.exp(1j*np.deg2rad(phiu)) # [kV]
i = np.sqrt(3) * I * np.exp(1j*np.deg2rad(phii)) # [kA]
S = u*i.conjugate()  # [MVA]
Y = i/u              # [S]
G,B = Y.real,Y.imag  # [s]

In [ ]:
PF_load = blob['PF_without_slack'].item()['loads']['LD1']
u_PF = np.sqrt(3) * PF_load['V'] * np.exp(1j*np.deg2rad(PF_load['phiu'])) # [kV]
i_PF = np.sqrt(3) * PF_load['I'] * np.exp(1j*np.deg2rad(PF_load['phii'])) # [kA]
S_PF = u_PF*i_PF.conjugate() # [MVA]
Y_PF = i_PF/u_PF             # [S]
assert np.abs(S_PF.real - PF_load['P']) < 1e-6
assert np.abs(S_PF.imag - PF_load['Q']) < 1e-6
print('G = {:.3f} S'.format(Y_PF.real))
print('B = {:.3f} S'.format(Y_PF.imag))

In [ ]:
fig,ax = plt.subplots(3, 1, figsize=(5,4), sharex=True)
ax[0].plot(time, Ul, 'k', lw=0.5)
ax[1].plot(time, I, 'k', lw=0.5)
phi = np.unwrap(np.deg2rad(phiu))
ylim = [phi.min(), phi.max()]
ax[2].plot(time, phi, 'r', lw=1, label=r'$\phi_U$')
phi = np.unwrap(np.deg2rad(phii))
ax[2].plot(time, phi, 'g', lw=1, label=r'$\phi_I$')
if phi.min() < ylim[0]: ylim[0] = phi.min()
if phi.max() > ylim[1]: ylim[1] = phi.max()
ylim = [np.pi*np.floor(ylim[0]/np.pi), np.pi*np.ceil(ylim[1]/np.pi)]
ax[2].legend(loc='best', frameon=False)
ticks = np.r_[ylim[0] : ylim[1]+0.1 : np.pi]
ax[2].set_ylim(ylim)
ax[2].set_yticks(ticks)
ax[2].set_yticklabels([f'{tick/np.pi:g}π' if tick != 0 else '0' for tick in ticks])
ax[2].set_xlabel('Time [s]')
ax[0].set_ylabel('Vl [kV]')
ax[1].set_ylabel('I [kA]')
ax[2].set_ylabel(r'$\phi$ [rad]')
sns.despine()
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(5,4), sharex=True)
ax[0].plot(time, S.real, 'k', lw=0.5)
ax[1].plot(time, S.imag, 'r', lw=0.5)
ax[1].set_xlabel('Time [s]')
ax[0].set_ylabel('P [MW]')
ax[1].set_ylabel('Q [MVAr]')
sns.despine()
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(5,4), sharex=True)
ax[0].plot(time, G, 'k', lw=0.5)
ax[0].plot(time[[0,-1]], Y_PF.real+np.zeros(2), 'r--', lw=2)
ax[1].plot(time, B, 'k', lw=0.5)
ax[1].plot(time[[0,-1]], Y_PF.imag+np.zeros(2), 'r--', lw=2)
ax[1].set_xlabel('Time [s]')
ax[0].set_ylabel('G [S]')
ax[1].set_ylabel('B [S]')
sns.despine()
fig.tight_layout()